In [88]:


import copy
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)


In [89]:
### target_files

cn_target_file='./new_cn_target'
cn_source_file='./new_cn_source'
vm_file='./new_vm'

In [90]:
### res_s + examples
cpu_per_cn = 56
#ram_per_cn = 512000
ram_per_cn = 493040
test_machine = 'site-liferay-10'
test_hypervisor = 'cn-d720.egron.tnx'

In [91]:
with open(vm_file, 'r') as f:
    vml = f.readlines()
    vml = [i.replace('\n','').split(',') for i in vml]
    
f.close()
with open(cn_target_file, 'r') as f:
    hpl = f.readlines()
    hpl = [i.replace('\n','').split(',') for i in hpl]
    
f.close()

In [92]:
def vm_dict_create(sl):
    'создаем словарь виртуальных машин'
    
    rd = {}
    for i in sl:
        if i[1]=='':
            crit='E'
        else:
            crit=i[1]
        rd[i[0]] = {'name': i[0],\
                 'cpu' : int(i[2]),\
                 'ram' : int(i[3]),\
                 'cn_name':i[4],\
                 'crit_rate':crit


        }
    return rd


def hyper_dict_create(c,r,hl,sl):
    'создаем  словарь гипервизоров'
    rd= {}
    for i in hl:
        h_vm_list = [s[0] for s in sl if s[4] == i[0]]
        rd[i[0]] = { 'name' : i[0],\
                         'cpu'  : c,\
                         'ram'  : r,\
                         'vms'  : h_vm_list,\
                         'vms_count' : len(h_vm_list)


            }
    return rd

def hyper_dict_resources_create(hd,vd):
    rd = copy.deepcopy(hd)
    for i in rd.keys():
        #print([vd[j]['ram'] for j in rd[i]['vms']] )
        sum_ram_perc = (rd[i]['ram'] - sum([vd[j]['ram'] for j in rd[i]['vms']]))/rd[i]['ram'] 
        sum_cpu_perc = (rd[i]['cpu'] - sum([vd[j]['cpu'] for j in rd[i]['vms']]))/rd[i]['cpu']
        rd[i]['perc_cpu'] = sum_cpu_perc
        rd[i]['perc_ram'] = sum_ram_perc
        rd[i]['free_cpu'] = (rd[i]['cpu'] - sum([vd[j]['cpu'] for j in rd[i]['vms']]))
        rd[i]['free_ram'] = (rd[i]['ram'] - sum([vd[j]['ram'] for j in rd[i]['vms']]))
    return rd, copy.deepcopy(rd)

In [93]:
def count_over (h,v):
    vm_vcpu_sum = sum([v[i]['cpu'] for i in v.keys()])
    vm_ram_sum = sum([v[i]['ram'] for i in v.keys()])
    hyper_vcpu_sum = sum([h[i]['cpu'] for i in h.keys()])
    hyper_ram_sum = sum([h[i]['ram'] for i in h.keys()])
    
    return [vm_vcpu_sum, vm_ram_sum], [hyper_vcpu_sum,hyper_ram_sum],\
        [vm_vcpu_sum / hyper_vcpu_sum, vm_ram_sum/hyper_ram_sum] 

In [94]:
def calculate_after_loc(v,h,hdictinuse,gdictinuse):
    ### Изменения во временных словарях после локации
    hdictinuse[h]['free_cpu'] = (hdictinuse[h]['free_cpu'] - vmd[v]['vcpu']) 
    hdictinuse[h]['perc_cpu'] = (hdictinuse[h]['free_cpu'] - vmd[v]['vcpu']) / cpu_per_cn
    hdictinuse[h]['free_ram'] = (hdictinuse[h]['free_ram'] - vmd[v]['ram']) 
    hdictinuse[h]['perc_ram'] = (hdictinuse[h]['free_ram'] - vmd[v]['ram']) / ram_per_cn
    hdictinuse[h]['vm_count'] = (hdictinuse[h]['vm_count'] + 1)
    gdictinuse[vmd[v]['groupid']]['az'][hpd[h]['az']] = gdictinuse[vmd[v]['groupid']]['az'][hpd[h]['az']] + 1 
    gdictinuse[vmd[v]['groupid']]['hyp'][h] = gdictinuse[vmd[v]['groupid']]['hyp'][h] + 1
    hdictinuse[h]['vms_ids'].append(v)
    if vmd[v]['crit_rate'] == 2:
        hdictinuse[h]['vm_critical'] = hdictinuse[h]['vm_critical'] + 1
    if vmd[v]['crit_rate'] == 1:
        hdictinuse[h]['vm_warning'] = hdictinuse[h]['vm_warning'] + 1

In [95]:
vmd = vm_dict_create(sl=vml)
hpd= hyper_dict_create(cpu_per_cn,ram_per_cn,hpl,vml)
hpd_res, hpd_res_in_use = hyper_dict_resources_create(hpd,vmd)
vm_res_total_list, hyper_res_total_list, over_total_list = count_over(hpd_res,vmd)

In [96]:
#vmd[test_machine],hpd[test_hypervisor],hpd_res[test_hypervisor],hpd_res_in_use[test_hypervisor]

In [97]:
l = [hpd_res[h]['perc_ram'] for h in hpd_res.keys()]
m = [hpd_res[h]['perc_cpu'] for h in hpd_res.keys()]



print("Всего машин:", len(vml))
print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
print("На одном ВУ CPU :", cpu_per_cn, ", RAM :", ram_per_cn, "MB")
print("Всего ВУ:", len(hpl))
print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

print("Забитость ВУ по RAM")
print("Самый забитый :", 1-min(l))
print("Наименее забитый :", 1-max(l))
print("Средняя загруженность :",  1 - sum(l)/len(l))
print('')
print("Перерасход vCPU относительно CPU ВУ ")
print("Самый забитый :", 1-min(m))
print("Наименее забитый :", 1-max(m))
print("Средняя загруженность :", 1 - sum(m)/len(m))


Всего машин: 1171
Необходимо машинам vCPU : 9345 , RAM : 23328448 MB
На одном ВУ CPU : 56 , RAM : 493040 MB
Всего ВУ: 66
Суммарно на ВУ : 3696 , RAM : 32540640 MB
Суммарная подписка по CPU : 2.528409090909091 , RAM : 0.7169019416950619
Забитость ВУ по RAM
Самый забитый : 1.3624533506409215
Наименее забитый : 0.06646113905565476
Средняя загруженность : 0.7174054351727563

Перерасход vCPU относительно CPU ВУ 
Самый забитый : 3.7142857142857144
Наименее забитый : 0.1428571428571429
Средняя загруженность : 2.5305735930735933


### Заполняем словарь hpd_in_use информацией о заполненности

In [98]:
hpd_res_in_use[test_hypervisor]

{'cpu': 56,
 'free_cpu': -128,
 'free_ram': 83440,
 'name': 'cn-d720.egron.tnx',
 'perc_cpu': -2.2857142857142856,
 'perc_ram': 0.16923576180431607,
 'ram': 493040,
 'vms': ['cia-sign-06',
  'cia-sign-08',
  'isurz-elasticsearch-07',
  'isurz-xml-storage-21',
  'pg-attribute-73',
  'pg-attribute-standby-07',
  'pg-attribute-standby-25',
  'pg-attribute-standby-89',
  'pkurp-worker-020',
  'pmo-db-fed-01',
  'ppoz-bus-command-04',
  'ppoz-bus-command-08',
  'ppoz-bus-data-08',
  'ppoz-bus-data-10',
  'ppoz-db-proc-gmp-03',
  'ppoz-db-proc-gmp-05',
  'ppoz-db-proc-pkurp-05',
  'ppoz-db-proc-popd-05',
  'ppoz-db-proc-ppoz-05',
  'ppoz-db-request-07',
  'ppoz-popd-process-04',
  'ppoz-process-core-03',
  'ppoz-process-core-05',
  'ppoz-service-external-03'],
 'vms_count': 24}

### Распределение


In [99]:
def find_best_hid_by_score(v,h_use):
    rd={}
    for h in h_use.keys(): 
       
        free_ram = h_use[h]['free_ram']
        free_cpu = h_use[h]['free_cpu']
        if free_ram < 0:
            continue
        score = count_score(free_cpu,free_ram)
        rd[h]={#'mply': score_mply,\
                              # 'score_az':score_az,\
                              # 'score_hyp':score_hyp, 
                               'score': score,\
                               'id' : h,\
                               'ram': h_use[h]['free_ram'],\
                               'cpu': h_use[h]['free_cpu'],\
                               'vm_count' : h_use[h]['vms']
                              }
    return rd

        
def count_score(pc,pr):
    
    res = (0.05 * pc + 0.5 * pr)/2

    ### score = 1.2 * az * 2.5 * hz * (1 - mw)
    ### Учитываем ресурсы
    score = res
    #print(res,az,hz,mw,score)
    return score
        

In [100]:
score_for_test = find_best_hid_by_score(test_machine,hpd_res_in_use)

In [101]:
scorint_list = [[i, score_for_test[i]['score'], score_for_test[i]['ram']] for i in score_for_test.keys() ]

In [102]:
sorted(scorint_list, key = lambda x: float(x[1]), reverse=True)

[['cn-db26.egron.tnx', 115069.2, 460272],
 ['cn-e624.egron.tnx', 85371.65, 341488],
 ['cn-dce2.egron.tnx', 70011.4, 280048],
 ['cn-dcaa.egron.tnx', 70011.1, 280048],
 ['cn-e4b8.egron.tnx', 62843.05, 251376],
 ['cn-da44.egron.tnx', 62842.75, 251376],
 ['cn-dc84.egron.tnx', 61819.05, 247280],
 ['cn-db38.egron.tnx', 60794.8, 243184],
 ['cn-e8ae.egron.tnx', 59770.7, 239088],
 ['cn-db3e.egron.tnx', 57722.6, 230896],
 ['cn-db32.egron.tnx', 55674.4, 222704],
 ['cn-dca6.egron.tnx', 49530.4, 198128],
 ['cn-dca8.egron.tnx', 47482.2, 189936],
 ['cn-dc9a.egron.tnx', 46458.75, 185840],
 ['cn-da32.egron.tnx', 46458.5, 185840],
 ['cn-e80c.egron.tnx', 44409.35, 177648],
 ['cn-dca4.egron.tnx', 43386.4, 173552],
 ['cn-e0dc.egron.tnx', 42361.85, 169456],
 ['cn-e8a8.egron.tnx', 42361.45, 169456],
 ['cn-da22.egron.tnx', 41337.7, 165360],
 ['cn-da1a.egron.tnx', 41337.65, 165360],
 ['cn-e0f6.egron.tnx', 38265.95, 153072],
 ['cn-ed8c.egron.tnx', 38265.7, 153072],
 ['cn-fe1a.egron.tnx', 37241.75, 148976],
 ['c

In [103]:
ram_test = [[i,hpd_res_in_use[i]['free_ram']]for i in hpd_res_in_use.keys()]

In [104]:
cn_test = sorted(ram_test, key = lambda x: int(x[1]))[:1][0][0]

In [105]:
cn_test

'cn-ee38.egron.tnx'

In [106]:
mean([vmd[j]['ram'] for j in hpd_res_in_use[cn_test]['vms']])

61067.63636363636